In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import transformers as ppb
import warnings
import os
import re
import gc
import torch
from tqdm import tqdm
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
gc.collect()

# Converting the sentences into tokens 

In [ ]:
train = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")

In [ ]:
train = reduce_mem_usage(train)
gc.collect()

In [ ]:
train['target'].value_counts()
#train['target'].value_counts()

In [ ]:
indexes = []
for f,i in enumerate(train['comment_text']):
    if len(i) > 512: 
        indexes.append(f)
gc.collect()        

In [ ]:
df = train.iloc[indexes]

In [ ]:
df

In [ ]:
df['toxic'] = np.where(df['target'] >= .5, 1, 0)

In [ ]:
df['toxic'].value_counts()

In [ ]:
train['toxic'] = np.where(train['target'] >= .5, 1, 0)

In [ ]:
train['toxic'].value_counts()

In [ ]:
gc.collect()

In [ ]:
train = train.drop(index= indexes)

In [ ]:
train.head(30)

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
gc.collect()

In [ ]:
del indexes
gc.collect()

In [ ]:
labels = train["target"] 

In [ ]:
 train['comment_text']= train['comment_text'].astype(str)

In [ ]:
 train

In [ ]:
tokenized = train['comment_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
gc.collect()

In [ ]:
del train, df
gc.collect()

In [ ]:
len(tokenized)

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
gc.collect()

In [ ]:
del max_len
gc.collect()


In [ ]:
print(np.array(padded).shape)
gc.collect()

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)
gc.collect()

In [ ]:
del tokenized
gc.collect()

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)


with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

In [ ]:
lr_clf.score(test_features, test_labels)